In [16]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision import models
import matplotlib.pyplot as plt


import numpy as np
import wandb
import pandas as pd
import cv2
from tqdm import tqdm
import argparse
from PIL import Image

from sklearn.metrics import f1_score, precision_score, accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [17]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [18]:
class Model_VGG(nn.Module):
    def __init__(self, model_type):
        super(Model_VGG, self).__init__()

        if model_type == "vgg_16":
            self.model_backbone = models.vgg16(pretrained=True)
        elif model_type == "vgg_19":
            self.model_backbone = models.vgg19(pretrained=True)
        self.model_backbone.classifier[6] = nn.Linear(4096, 10)

    def forward(self, x):
        x = self.model_backbone(x)
        return x

In [19]:
class Digit_Dataset(Dataset):
    def __init__(self, df_data, transforms=None):
        self.data = df_data
        self.transforms = transforms

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = self.data.iloc[idx, 1:].values.reshape(28, 28).astype(np.uint8)
        # 3 channels
        img = np.stack((img,) * 3, axis=-1)
        label = self.data.iloc[idx, 0]
        if self.transforms:
            img = self.transforms(img)
        return img, label


In [20]:
class Training_Model:
    def __init__(self, model, optimizer, criterion, dict_dataloader):
        self.model = model.to(device)
        self.optimizer = optimizer
        self.criterion = criterion
        self.train_loader = dict_dataloader["train_loader"]
        self.val_loader = dict_dataloader["val_loader"]
        self.best_f1_score = 0

    def train(self, epochs):
        print("============TRAINING START {}============".format(epochs))
        self.model.train()
        total_loss = 0
        preds, targets = [], []
        for batch_idx, (data, target) in enumerate(tqdm(self.train_loader)):
            data = data.to(device)
            target = target.to(device)
            self.optimizer.zero_grad()
            output = self.model(data)
            loss = self.criterion(output, target)
            loss.backward()
            self.optimizer.step()
            total_loss += loss.item()
            _, predicted = torch.max(output.data, 1)
            preds += predicted.tolist()
            targets += target.tolist()

        total_loss = round(total_loss / len(self.train_loader), 4)
        precision_scr = round(
            precision_score(targets, preds, average="macro"),
            4,
        )
        accuracy_scr = round(
            accuracy_score(targets, preds),
            4,
        )
        train_f1_scr = round(
            f1_score(targets, preds, average="macro"),
            4,
        )
        print(
            "train_loss: {}, train_precision: {}, train_accuracy: {}, train_f1_score: {}".format(
                total_loss, precision_scr, accuracy_scr, train_f1_scr
            )
        )
        wandb.log(
            {
                "train_loss": total_loss,
                "train_precision_score": precision_scr,
                "train_accuracy_score": accuracy_scr,
                "train_f1_score": train_f1_scr,
            }
        )

    def validation(self, epochs):
        print("============VAL START {}============".format(epochs))
        self.model.eval()
        total_loss = 0
        preds, targets = [], []
        for batch_idx, (data, target) in enumerate(tqdm(self.val_loader)):
            data = data.to(device)
            target = target.to(device)

            with torch.no_grad():
                output = self.model(data)
                loss = self.criterion(output, target)
                total_loss += loss.item()
                _, predicted = torch.max(output.data, 1)

                preds += predicted.tolist()
                targets += target.tolist()
        total_loss = round(total_loss / len(self.val_loader), 4)
        precision_scr = round(precision_score(targets, preds, average="macro"), 4)
        accuracy_scr = round(accuracy_score(targets, preds), 4)
        val_f1_scr = round(f1_score(targets, preds, average="macro"), 4)
        print(
            "val_loss: {}, val_precision: {}, val_accuracy: {}, val_f1_score: {}".format(
                total_loss, precision_scr, accuracy_scr, val_f1_scr
            )
        )
        wandb.log(
            {
                "val_loss": total_loss,
                "val_precision_score": precision_scr,
                "val_accuracy_score": accuracy_scr,
                "val_f1_score": val_f1_scr,
            }
        )
        if val_f1_scr > self.best_f1_score and val_f1_scr > 0.9:
            self.best_f1_score = val_f1_scr
            torch.save(
                self.model.state_dict(),
                "/kaggle/working/{}_f1_{}.pt".format("vgg_19", val_f1_scr),
            )
#             torch.jit.save(
#                 torch.jit.script(self.model),
#                 self.model.state_dict(),
#                 "/kaggle/working/{}_jit_f1_{}.pt".format("vgg_16", val_f1_scr),
#             )

In [21]:
def get_loader(csv_file, transforms, batch_size=64):
    dataset = Digit_Dataset(csv_file, transforms)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return loader


def get_input():
    parser = argparse.ArgumentParser()
    parser.add_argument("--model_type", type=str, default="vgg_19")
    parser.add_argument("--batch_size", type=int, default=8)
    parser.add_argument("--epochs", type=int, default=20)
    parser.add_argument("--lr", type=float, default=0.001)
    parser.add_argument("--seed", type=int, default=10)
    parser.add_argument("--weight_decay", type=float, default=0.0001)
    args = parser.parse_args()
    return args


def config_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

In [22]:
config_seed(10)

batch_size = 128
epochs = 50
lr = 0.001
weight_decay = 0.0001

data = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")

train_data, val_data = train_test_split(data, test_size=0.2, random_state=10)

transforms = transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5,), std=(0.5,)),
    ]
)

train_loader = get_loader(train_data, transforms=transforms, batch_size=batch_size)
val_loader = get_loader(val_data, transforms=transforms, batch_size=batch_size)

dict_dataloader = {"train_loader": train_loader, "val_loader": val_loader}
model_type = "vgg_19"
model = Model_VGG(model_type)
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()

wandb.init(
    project="Digit_Recognizer",
    group="VGG",
    name="{}_{}_{}_{}".format("vgg_19", batch_size, epochs, lr),
)

wandb.watch(model)

training_model = Training_Model(
    model=model,
    optimizer=optimizer,
    criterion=criterion,
    dict_dataloader=dict_dataloader,
)

for epoch in range(epochs):
    training_model.train(epoch)
    training_model.validation(epoch)

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


  0%|          | 0.00/548M [00:00<?, ?B/s]

wandb: Currently logged in as: nabang1010_work (lban_nlp_vinbigdata). Use `wandb login --relogin` to force relogin


============TRAINING START 0============


100%|██████████| 263/263 [00:57<00:00,  4.59it/s]


...

val_loss: 0.0478, val_precision: 0.9927, val_accuracy: 0.9929, val_f1_score: 0.9927
============TRAINING START 49============


100%|██████████| 263/263 [00:48<00:00,  5.44it/s]


train_loss: 0.0167, train_precision: 0.9969, train_accuracy: 0.9968, train_f1_score: 0.9969
============VAL START 49============


100%|██████████| 66/66 [00:07<00:00,  8.58it/s]


val_loss: 0.1101, val_precision: 0.9925, val_accuracy: 0.9926, val_f1_score: 0.9925


In [23]:
model_type = "vgg_19"
model = Model_VGG(model_type)

In [24]:
model.load_state_dict(torch.load("/kaggle/working/vgg_19_f1_0.9927.pt"))

<All keys matched successfully>

In [25]:
model.eval()

Model_VGG(
  (model_backbone): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
 

In [26]:
model = model.to(device)

In [27]:
DF_test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")
DF_test

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
from torchvision import transforms
DF_submission = pd.DataFrame(columns=["ImageId", "Label"])
transforms = transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5,), std=(0.5,)),
    ]
)
for row in tqdm(range(DF_test.shape[0])):
    img = DF_test.iloc[row, :].values
    img = img.reshape(28, 28).astype(np.uint8)
    # 3 channel
    img = np.stack((img,) * 3, axis=-1)
    img = transforms(img)
    
    img = img.to(device)
    img = img.unsqueeze(0)
    with torch.no_grad():
        output = model(img)
        _, predicted = torch.max(output.data, 1)
        DF_test.iloc[row, :] = predicted.item()
        DF_submission = DF_submission.append( {"ImageId": row+1, "Label": predicted.item()}, ignore_index=True)
DF_submission.to_csv("/kaggle/working/vgg_19_f1_50_0.9927_submission.csv", index=False)


100%|██████████| 28000/28000 [02:50<00:00, 164.35it/s]
